# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.abspath('..'))

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
def get_local_llm():
    print("Loading Ollama")
    return Ollama(model="llama3.1")

def get_azure_llm():
    os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
    os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
    os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
    print("Loading Azure OpenAI")
    return AzureChatOpenAI(
        deployment_name=os.getenv("AZURE_DEPLOYMENT__GPT_4"),
    )

def get_llm(run_local):
    if run_local == "True" or run_local == "true":
        return get_local_llm()
    else:
        return get_azure_llm()


run_local = os.getenv("RUN_LOCAL")
llm = get_llm(run_local)

Loading Azure OpenAI


In [4]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'Meta',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (Meta). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data for Meta, I need to delegate specific tasks to the right team members. The Data Analyst should start by collecting and analyzing the current and historical market data for Meta, using statistical modeling and machine learning to identify trends.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Collect and analyze market data", "context": "The task involves gathering current and historical market data for the stock of Meta. Use statistical modeling and machine learning techniques to identify trends and predict market movements. Focus on identifying any patterns that could indicate significant market opportunities or threa

KeyboardInterrupt: 

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for AAPL

Introduction:
In this report, we will analyze the risks associated with the proposed trading strategies and execution plans for AAPL (Apple Inc.). Due to the unavailability of detailed execution plans from the Trading Strategy Developer, alternative sources were explored to gather information on potential strategies for trading AAPL. The following analysis provides insights into potential risks and suggests mitigation strategies to enhance the overall performance and success of the trading strategies.

Analysis of Potential Risks:
1. Scalping Strategy:
The Scalping strategy involves executing multiple trades within a short time frame to capitalize on small price differentials. Risks associated with this strategy include:
- High frequency of trades may increase transaction costs.
- Market volatility can lead to rapid price changes, affecting profitability.
- Scalping requires quick decision-making, which may result in impulsive trading behavior.

Mitigation Strategy:
Implement strict risk management protocols to control transaction costs and minimize losses. Utilize technical analysis tools to identify potential entry and exit points accurately. Develop a disciplined approach to avoid emotional trading decisions.

2. Swing Trading Strategy:
Swing trading aims to capture short to medium-term price movements in AAPL. Risks associated with this strategy include:
- Market fluctuations may lead to unexpected price reversals.
- Holding positions overnight exposes traders to overnight risk.
- Inadequate analysis of fundamental and technical factors may result in poor trade decisions.

Mitigation Strategy:
Conduct thorough fundamental and technical analysis to identify potential swing trading opportunities. Set stop-loss orders to limit potential losses and protect capital. Monitor market trends closely to adjust trading strategies accordingly.

3. Options Trading Strategy:
Options trading involves leveraging derivatives contracts to speculate on AAPL's price movements. Risks associated with this strategy include:
- Limited time frame for options contracts may lead to time decay.
- Market volatility can impact the value of options contracts.
- Lack of understanding of options pricing model may result in mispriced contracts.

Mitigation Strategy:
Educate traders on options pricing models and strategies to make informed decisions. Implement risk management strategies such as hedging to protect against adverse price movements. Regularly review options positions to adjust strategies based on changing market conditions.

Conclusion:
In conclusion, the proposed trading strategies for AAPL present various risks that traders need to consider when executing trades. By conducting a thorough risk analysis and implementing appropriate mitigation strategies, traders can enhance their chances of success and achieve profitable outcomes in trading AAPL. It is essential to stay informed about market trends, utilize risk management tools effectively, and adapt trading strategies based on changing market conditions to optimize trading performance.